In [66]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline
import seaborn as sns

In [67]:
df_train=pd.read_csv("train.csv")
df_test=pd.read_csv("test.csv")

In [68]:
col = ["MSZoning","LotShape","HouseStyle","Neighborhood","Exterior1st","Exterior2nd","Foundation","MasVnrType","ExterQual","BsmtQual","BsmtFinType1","HeatingQC","KitchenQual","FireplaceQu","GarageType","GarageFinish"]

In [69]:
x_train=df_train.loc[:,col]
x_test=df_test.loc[:,col]

In [70]:
x_test.isnull().sum().sort_values(ascending=False)

FireplaceQu     730
GarageFinish     78
GarageType       76
BsmtQual         44
BsmtFinType1     42
MasVnrType       16
MSZoning          4
KitchenQual       1
Exterior2nd       1
Exterior1st       1
HeatingQC         0
ExterQual         0
Foundation        0
Neighborhood      0
HouseStyle        0
LotShape          0
dtype: int64

In [71]:
x_train.loc[x_train.isnull()["GarageFinish"],:] = x_train.loc[x_train.isnull()["GarageFinish"],:].fillna(value="None")
x_train.loc[x_train.isnull()["GarageType"],:] = x_train.loc[x_train.isnull()["GarageType"],:].fillna(value="None")
x_train.loc[x_train.isnull()["FireplaceQu"],:] = x_train.loc[x_train.isnull()["FireplaceQu"],:].fillna(value="None")
x_train.loc[x_train.isnull()["BsmtQual"],:] = x_train.loc[x_train.isnull()["BsmtQual"],:].fillna(value="None")
x_train.loc[x_train.isnull()["BsmtFinType1"],:] = x_train.loc[x_train.isnull()["BsmtFinType1"],:].fillna(value="None")
x_train.loc[x_train.isnull()["MasVnrType"],:] = x_train.loc[x_train.isnull()["MasVnrType"],:].fillna(value="None")

In [72]:
x_test.loc[x_test.isnull()["KitchenQual"],:] = x_test.loc[x_test.isnull()["KitchenQual"],:].fillna(value="None")
x_test.loc[x_test.isnull()["GarageFinish"],:] = x_test.loc[x_test.isnull()["GarageFinish"],:].fillna(value="None")
x_test.loc[x_test.isnull()["GarageType"],:] = x_test.loc[x_test.isnull()["GarageType"],:].fillna(value="None")
x_test.loc[x_test.isnull()["FireplaceQu"],:] = x_test.loc[x_test.isnull()["FireplaceQu"],:].fillna(value="None")
x_test.loc[x_test.isnull()["BsmtQual"],:] = x_test.loc[x_test.isnull()["BsmtQual"],:].fillna(value="None")
x_test.loc[x_test.isnull()["BsmtFinType1"],:] = x_test.loc[x_test.isnull()["BsmtFinType1"],:].fillna(value="None")
x_test.loc[x_test.isnull()["MasVnrType"],:] = x_test.loc[x_test.isnull()["MasVnrType"],:].fillna(value="None")
x_test.loc[x_test.isnull()["Exterior2nd"],:] = x_test.loc[x_test.isnull()["Exterior2nd"],:].fillna(value="None")
x_test.loc[x_test.isnull()["Exterior1st"],:] = x_test.loc[x_test.isnull()["Exterior1st"],:].fillna(value="None")
x_test.loc[x_test.isnull()["MSZoning"],:] = x_test.loc[x_test.isnull()["MSZoning"],:].fillna(value="None")

In [73]:
add_x = pd.concat([x_train,x_test])

In [74]:
one_hot = pd.get_dummies(add_x)

In [75]:
train_onehot=one_hot.iloc[0:1460,:]

In [76]:
test_onehot=one_hot.iloc[1460:2920,:]

-----

In [77]:
from sklearn.preprocessing import MinMaxScaler

In [78]:
x_data =df_train.loc[:,["OverallQual","YearBuilt","YearRemodAdd","MasVnrArea","TotalBsmtSF","1stFlrSF","GrLivArea","FullBath","TotRmsAbvGrd","Fireplaces","GarageCars","GarageArea"]]

In [79]:
x_data.isnull().sum().sort_values(ascending=False)

MasVnrArea      8
GarageArea      0
GarageCars      0
Fireplaces      0
TotRmsAbvGrd    0
FullBath        0
GrLivArea       0
1stFlrSF        0
TotalBsmtSF     0
YearRemodAdd    0
YearBuilt       0
OverallQual     0
dtype: int64

In [80]:
x_data.loc[x_data.isnull()["MasVnrArea"],:] = x_data.loc[x_data.isnull()["MasVnrArea"],:].fillna(value=x_data["MasVnrArea"].mean())

In [81]:
x_data = pd.concat([train_onehot,x_data], axis=1)

In [82]:
x_data = MinMaxScaler().fit_transform(x_data)

In [83]:
y_data = df_train.iloc[:,-1]

In [84]:
y_data = y_data.values
y_data=y_data.reshape(-1,1)

In [85]:
print(x_data.shape)
print(y_data.shape)

(1460, 141)
(1460, 1)


In [86]:
X = tf.placeholder(tf.float32, shape=[None,141])
Y = tf.placeholder(tf.float32, shape=[None,1])


W = tf.Variable(tf.random_normal([141,1],name='weight'))
b = tf.Variable(tf.random_normal([1],name='bias'))

hypothesis = tf.matmul(X, W) + b

cost = tf.reduce_mean(tf.square(hypothesis-Y))

lamda = 0.001 * tf.reduce_sum(tf.square(W))
cost = cost + lamda


optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.1)
train = optimizer.minimize(cost)

In [87]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for step in range(200001):
    cost_val, hy_val, _ = sess.run(
        [cost,hypothesis,train],
        feed_dict={X:x_data, Y:y_data})
    if step % 2000 == 0:
        print(step,"Cost : ",cost_val,
             "\nPreadiction\n",hy_val)

0 Cost :  39038726000.0 
Preadiction
 [[-0.68752617]
 [ 0.49385458]
 [-1.9191344 ]
 ...
 [-2.517138  ]
 [-1.3191991 ]
 [ 8.109603  ]]
2000 Cost :  940122200.0 
Preadiction
 [[207912.72]
 [201602.12]
 [211548.69]
 ...
 [278947.6 ]
 [131762.27]
 [127217.41]]
4000 Cost :  937135940.0 
Preadiction
 [[207402.31]
 [202352.47]
 [211536.6 ]
 ...
 [279542.1 ]
 [132036.53]
 [127166.93]]
6000 Cost :  936372100.0 
Preadiction
 [[207267.25]
 [202330.23]
 [211636.1 ]
 ...
 [279850.1 ]
 [132132.45]
 [127205.08]]
8000 Cost :  936134660.0 
Preadiction
 [[207226.64]
 [202283.22]
 [211690.73]
 ...
 [280019.75]
 [132172.61]
 [127227.43]]
10000 Cost :  936057100.0 
Preadiction
 [[207212.39]
 [202252.02]
 [211720.78]
 ...
 [280114.72]
 [132193.16]
 [127237.09]]
12000 Cost :  936030500.0 
Preadiction
 [[207206.45]
 [202233.75]
 [211738.  ]
 ...
 [280169.75]
 [132205.3 ]
 [127240.67]]
14000 Cost :  936020740.0 
Preadiction
 [[207203.58]
 [202223.27]
 [211748.33]
 ...
 [280203.06]
 [132212.94]
 [127241.69]]
16

---

In [88]:
x_data =df_test.loc[:,["OverallQual","YearBuilt","YearRemodAdd","MasVnrArea","TotalBsmtSF","1stFlrSF","GrLivArea","FullBath","TotRmsAbvGrd","Fireplaces","GarageCars","GarageArea"]]

In [89]:
x_data.loc[x_data.isnull()["MasVnrArea"],:] = x_data.loc[x_data.isnull()["MasVnrArea"],:].fillna(value=x_data["MasVnrArea"].mean())
x_data.loc[x_data.isnull()["GarageCars"],:] = x_data.loc[x_data.isnull()["GarageCars"],:].fillna(value=x_data["GarageCars"].mean())
x_data.loc[x_data.isnull()["GarageArea"],:] = x_data.loc[x_data.isnull()["GarageArea"],:].fillna(value=x_data["GarageArea"].mean())
x_data.loc[x_data.isnull()["TotalBsmtSF"],:] = x_data.loc[x_data.isnull()["TotalBsmtSF"],:].fillna(value=x_data["TotalBsmtSF"].mean())

In [90]:
x_data_test = pd.concat([test_onehot,x_data], axis=1)

In [91]:
x_data_test = MinMaxScaler().fit_transform(x_data_test)

In [92]:
x_data_test.shape

(1459, 141)

----

In [93]:
sess.run(hypothesis, feed_dict={X:x_data_test})

array([[102997.984],
       [140422.83 ],
       [165276.62 ],
       ...,
       [144747.81 ],
       [ 95914.266],
       [188761.62 ]], dtype=float32)

In [94]:
sample = pd.read_csv("sample_submission.csv")

In [95]:
sample['SalePrice']=sess.run(hypothesis, feed_dict={X:x_data_test})

In [96]:
sample.to_csv("sample6.csv",index=False)

---

In [97]:
def log_rmse(predict_y, y_val):
    if predict_y.shape[0] != y_val.shape[0]:
        error_msg = "not matched length of arrays"
        return error_msg
    else:
        RMSE_log = np.sqrt(((np.log(predict_y) - np.log(y_val))**2).mean())
        return RMSE_log

In [98]:
y_train =df_train.iloc[:,-1]

In [99]:
log_rmse(hy_val.reshape(-1,), y_train)

0.1428313568299162